In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.ticker
import numpy as np
import inmeta

In [ ]:
logdir_prefix = "../logdir/gcloud"

logdirs = dict(fourier="fourier.icml",
               spherical="spherical.icml",
               school="school.icml",
               low_rank="fourier.low-rank.icml",
               subspace="subspace.icml")

logdirs = {k: os.path.join(logdir_prefix, d) 
           for k, d in logdirs.items()}

In [ ]:
fourier = inmeta.BatchedExperiment.from_logdir(logdirs["fourier"])
ntasks_fig, task_size_fig = fourier.plot()
ntasks_fig.axes[0].get_legend().remove()

plt.legend(
  ["Biased Regression",
   "EM Learner",
   "Linear Regression (All)",
   "Linear Regression (Task)",
   "Known Covariance Learner"], 
  loc="best");

for fname, fig in {"ntasks": ntasks_fig,
                   "task-size": task_size_fig}.items():
  fname = os.path.join(logdirs["fourier"], f"{fname}.pdf")
  fig.savefig(fname)

In [ ]:
def plot_task_test(exp, task_index=0, ntasks=None, task_size=None, 
                   learners=("em", "biased-regression")):
  adaptation_indices = np.arange(
      exp.runner.split.adaptation.features.shape[1])
  test_indices = (exp.runner.split.adaptation.features.shape[1]
                  + np.arange(exp.runner.split.test.features.shape[1]))
    
  exp.source.plot(1, task_index, adaptation_indices)
  exp.source.scatter_samples(1, task_index, test_indices, marker='.', s=10)

  train = inmeta.task.subset_samples(*exp.runner.split.train, ntasks, task_size)
  xs, features = exp.source.xs_features(1, task_index, test_indices)
  adapt, test = exp.runner.split.adaptation, exp.runner.split.test
  adapt = inmeta.task.Samples(adapt.features[task_index], adapt.labels[task_index])
  test = inmeta.task.Samples(test.features[task_index], test.labels[task_index])
  mses = dict()
  for key in sorted(learners if learners is not None else exp.learners):
    exp.learners[key].fit(*train)
    plot_preds = exp.learners[key].predict(*adapt, features)
    plt.plot(xs, plot_preds, label=key)
    mse_preds = exp.learners[key].predict(*adapt, test.features)
    mses[key] = inmeta.experiment.mse(labels=test.labels, preds=mse_preds)
  plt.legend()
  return mses

In [ ]:
for task_index in [0,1]:
  for ntasks in [10, 50, 100]:
    fig = plt.figure()
    plt.title(fr"$n={ntasks}$")
    plot_task_test(fourier.experiments[0], task_index, ntasks=ntasks, task_size=10, 
                   learners=("em", "biased-regression"))
    if task_index == 0 and ntasks == 10:
      plt.legend([r"Task Function (without label noise)", 
                  "Biased Regression",
                  "EM Learner"])
    else:
      plt.gca().get_legend().remove()
    plt.grid()
    fname = os.path.join(logdirs["fourier"], 
                         f"example-task-{task_index}-ntasks-{ntasks}.pdf")
    fig.savefig(fname)

In [ ]:
low_rank = inmeta.BatchedExperiment.from_logdir(logdirs["low_rank"])
ntasks_fig, task_size_fig = low_rank.plot(plot_fn=inmeta.plot.semilogy_mean_std)
ntasks_fig.axes[0].get_legend().remove()

for fig in (ntasks_fig, task_size_fig):
  fig.axes[0].set_yticks(np.geomspace(1.5, 40, 10))
  fig.axes[0].get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())

plt.legend(
  ["Biased Regression",
   "EM Learner",
   "Linear Regression (All)",
   "Linear Regression (Task)",
   "Known Covariance Learner",
   "Oracle Representation"], 
  loc="best");


for fname, fig in {"ntasks": ntasks_fig,
                   "task-size": task_size_fig}.items():
  fname = os.path.join(logdirs["low_rank"], f"{fname}.png")
  fig.savefig(fname)

In [ ]:
fig = inmeta.BatchedExperiment.from_logdir(logdirs["subspace"]).plot_ntasks(
  plot_fn=inmeta.plot.plot_mean_std)
fig.axes[0].set_ylabel(rf"$\sin(\theta)$");

plt.legend(["EM Learner", "MoM Representation"])

fname = os.path.join(logdirs["subspace"], "subspace.png")
fig.savefig(fname)

In [ ]:
spherical = inmeta.BatchedExperiment.from_logdir(logdirs["spherical"])
ntasks_fig, task_size_fig = spherical.plot(plot_fn=inmeta.plot.plot_mean_std)
ntasks_fig.axes[0].get_legend().remove()

plt.legend([
   "Biased Regression",
   "EM Learner",
   "Linear Regression (All)",
   "Linear Regression (Task)",
   "Known Covariance Learner"], 
  loc="best");

for fname, fig in {"ntasks": ntasks_fig,
                   "task-size": task_size_fig}.items():
  fname = os.path.join(logdirs["spherical"], f"{fname}.pdf")
  fig.savefig(fname)

In [ ]:
school = inmeta.BatchedExperiment.from_logdir(logdirs["school"])
ntasks_fig, _ = school.plot(plot_fn=inmeta.plot.semilogy_mean_std)
ntasks_fig.axes[0].get_legend().remove()

plt.legend([
   "Biased Regression",
   "EM Learner",
   "Linear Regression (All)",
   "Linear Regression (Task)"],
  loc="best");

for fname, fig in {"ntasks": ntasks_fig}.items():
  fname = os.path.join(logdirs["school"], f"{fname}.pdf")
  fig.savefig(fname)